In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
FINETUNE_MODEL = "./gemma-2b-it-qa-ko" # "google/gemma-2b-it", "./gemma-2b-it-qa-ko"
TOKENIZER = "./gemma-2b-it-tokenizer"

model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0}, push_to_hub=False)
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER, add_special_tokens=True, push_to_hub=False)

In [ ]:
query = '''
경조사 지원 규정
1. 적용대상
계약직 사원을 포함한 임직원(고문 및 용역은 사장이 별도로 결정)
2. 경조사 지원기준
경사의 경우, 본인 결혼 시 5일의 휴가와 50만 원의 금액, 화환이 지급됩니다.
자녀 결혼 시 1일의 휴가와 30만 원의 금액, 화환이 지급됩니다.
형제자매 결혼 시 1일의 휴가와 10만 원의 금액이 지급됩니다.
자녀 출산(배우자) 시 10일의 휴가와 10만 원의 금액이 지급되며, 1회 분할 사용이 가능합니다.
부모 회갑 시 1일의 휴가와 20만 원의 금액이 지급됩니다.
배우자 부모 회갑 시 1일의 휴가와 20만 원의 금액이 지급됩니다.
부모 고희(칠순) 시 1일의 휴가와 30만 원의 금액이 지급됩니다.
배우자 부모 고희 시 1일의 휴가와 30만 원의 금액이 지급됩니다.
조사의 경우, 본인 사망 시 5일의 휴가와 100만 원의 금액, 조화가 지급됩니다.
배우자 사망 시 5일의 휴가와 100만 원의 금액, 조화가 지급됩니다.
부모 사망 시 5일의 휴가와 100만 원의 금액, 조화가 지급됩니다.
자녀 사망 시 5일의 휴가와 100만 원의 금액, 조화가 지급됩니다.
배우자 부모 사망 시 5일의 휴가와 50만 원의 금액, 조화가 지급됩니다.
형제자매 사망 시 3일의 휴가와 30만 원의 금액, 조화가 지급됩니다.
조부모/외조부모 사망 시 3일의 휴가와 20만 원의 금액이 지급됩니다.

1. 목적
이 규정은 주식회사 모코엠시스에 근무하는 직원의 급여에 관한 제반사항을 정하여 합리적인 급여관리를 실행함을 목적으로 한다.
2. 적용범위
직원의 급여는 관계법령, 기타 별도로 정한 것을 제외하고는 이 규정이 정하는 바에 따른다.
3. 급여의 구성
3.1 급여는 기본급, 제수당, 상여금, 퇴직금으로 구분한다.
3.2 일반적으로 월급여라 함은 기본급과 제수당을 합한 금액을 말한다.
4. 급여조정
급여의 인상은 년 1회 1월 1일부로 실시함을 원칙으로 한다.

Q: 부모가 돌아가셨을때 유급휴가는 며칠?
'''

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
messages = [
    {
        "role": "user",
        "content": "{}".format(query)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(prompt)

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

In [ ]:
outputs